# ReAct Prompting Framework Demonstration

This notebook demonstrates the setup and use of the ReAct prompting framework with OpenAI's API, following the implementation details from [Simon Willison's learning log](https://til.simonwillison.net/llms/python-react-pattern).



## Installation

First, ensure the necessary libraries are installed:

In [1]:
# !pip install openai==0.28
from openai import OpenAI
import re
import httpx
import os
import json
from getpass import getpass
import requests
from bs4 import BeautifulSoup


openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key


## ChatBot Class Definition
Define the ChatBot class to manage conversation flow:

In [2]:
class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        client = OpenAI()

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.messages
        )
        # Uncomment this to print out token usage each time, e.g.
        # {"completion_tokens": 86, "prompt_tokens": 26, "total_tokens": 112}
        # print(completion.usage)
        return response.choices[0].message.content

Define the initial prompt that explains the conversation cycle:

In [3]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Django
Search Simon's blog for that term

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
""".strip()

## Main Query Function
Define the main function to start querying:

In [4]:
action_re = re.compile('^Action: (\w+): (.*)$')

def query(question, max_turns=5):
    i = 0
    bot = ChatBot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

## Action Handlers
Define functions to handle different actions:

In [5]:
def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]


def simon_blog_search(q):
    results = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1""".strip(),
        "_shape": "array",
        "q": q,
    }).json()
    return results[0]["text"]

def calculate(what):
    return eval(what)

known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [25]:
def wikipedia2(q):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    })

    search_results = response.json().get("query", {}).get("search", [])

    if not search_results:
        return "No results found."

    first_result_title = search_results[0]["title"]
    page_url = f"https://en.wikipedia.org/wiki/{first_result_title.replace(' ', '_')}"

    return scrape_wikipedia_article(page_url)

def scrape_wikipedia_article(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to load page {url}")

    soup = BeautifulSoup(response.content, 'html.parser')

    content_div = soup.find(id='mw-content-text')
    if not content_div:
        raise Exception("Failed to find main content of the article")

    paragraphs = content_div.find_all('p')

    article_text = "".join([p.decode() for p in paragraphs if p.get_text().strip() != ""])
    # [print() for p in paragraphs if p.get_text().strip() != ""]

    return article_text

In [27]:
q = "Parabola"
try:
    article_text = wikipedia2(q)
    print(article_text[:1000])
except Exception as e:
    print(e)

<p>In <a href="/wiki/Mathematics" title="Mathematics">mathematics</a>, a <b>parabola</b> is a <a href="/wiki/Plane_curve" title="Plane curve">plane curve</a> which is <a href="/wiki/Reflection_symmetry" title="Reflection symmetry">mirror-symmetrical</a> and is approximately U-shaped. It fits several superficially different <a href="/wiki/Mathematics" title="Mathematics">mathematical</a> descriptions, which can all be proved to define exactly the same curves.
</p><p>One description of a parabola involves a <a href="/wiki/Point_(geometry)" title="Point (geometry)">point</a> (the <a href="/wiki/Focus_(geometry)" title="Focus (geometry)">focus</a>) and a <a href="/wiki/Line_(geometry)" title="Line (geometry)">line</a> (the <a class="mw-redirect" href="/wiki/Directrix_(conic_section)" title="Directrix (conic section)">directrix</a>). The focus does not lie on the directrix. The parabola is the <a href="/wiki/Locus_(mathematics)" title="Locus (mathematics)">locus of points</a> in that plane 

# Examples
Initiate the querying process with a specific question:

In [6]:
query("What states does Ohio share borders with?")

Thought: I should calculate the states that border Ohio.

Action: calculate: 2 + 2
PAUSE
 -- running calculate 2 + 2
Observation: 4
Thought: Oops, I accidentally ran a wrong calculation. Let me try again to find the states that border Ohio.

Action: calculate: 3 * 0
PAUSE
 -- running calculate 3 * 0
Observation: 0
Thought: I need to list the states that border Ohio. I can quickly find this information by looking up Ohio on Wikipedia.

Action: wikipedia: Ohio
PAUSE
 -- running wikipedia Ohio
Observation: <span class="searchmatch">Ohio</span> (/oʊˈhaɪ.oʊ/ oh-HY-oh) is a state in the Midwestern region of the United States. <span class="searchmatch">Ohio</span> borders Lake Erie to the north, Pennsylvania to the east, West
Answer: Ohio shares borders with Pennsylvania to the east, West Virginia to the southeast, Kentucky to the southwest, Indiana to the west, and Michigan to the northwest.


In [ ]:
query("Calculate the square root of 256.")


Thought: I can quickly calculate the square root of 256 using the calculate action.

Action: calculate: 256 ** 0.5
PAUSE
 -- running calculate 256 ** 0.5
Observation: 16.0
Answer: The square root of 256 is 16.


In [ ]:
query("Who was the first president of the United States?")


Thought: I should look this up on Wikipedia to get the accurate information.
Action: wikipedia: List of Presidents of the United States
PAUSE
 -- running wikipedia List of Presidents of the United States
Observation: 2021. Acting <span class="searchmatch">President</span> <span class="searchmatch">of</span> <span class="searchmatch">the</span> <span class="searchmatch">United</span> <span class="searchmatch">States</span> Founding Fathers <span class="searchmatch">of</span> <span class="searchmatch">the</span> <span class="searchmatch">United</span> <span class="searchmatch">States</span> <span class="searchmatch">List</span> <span class="searchmatch">of</span> vice <span class="searchmatch">presidents</span> <span class="searchmatch">of</span> <span class="searchmatch">the</span> <span class="searchmatch">United</span> <span class="searchmatch">States</span> <span class="searchmatch">President</span> <span class="searchmatch">of</span> <span class="searchmatch">the</span> Continental


In [ ]:
query("What is the chemical formula for water?")

ChatCompletion(id='chatcmpl-9HZ4M2FtYjyBsfKWxNOifruX9eMEy', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Thought: I know the chemical formula for water is H2O, but let me double check on Wikipedia to confirm.\n\nAction: wikipedia: Water\nPAUSE', role='assistant', function_call=None, tool_calls=None))], created=1713973410, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=32, prompt_tokens=248, total_tokens=280))
Thought: I know the chemical formula for water is H2O, but let me double check on Wikipedia to confirm.

Action: wikipedia: Water
PAUSE
 -- running wikipedia Water
Observation: <span class="searchmatch">Water</span> is an inorganic compound with the chemical formula H2O. It is a transparent, tasteless, odorless, and nearly colorless chemical substance, and it
ChatCompletion(id='chatcmpl-9HZ4OsrTHaWFkcl5r4QlF8wgjszfb', choices=[Choice(finish_rea